In [1]:
from IPython.display import display, HTML

import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from collections import namedtuple
import itertools as iter

Result = namedtuple('Result', ['supply', 'demand', 'price', 'welfare'])

import scipy.optimize
from mosek.fusion import Model, Domain, Expr, ObjectiveSense


In [2]:
nodes = ['B1']
nnodes = len(nodes)

In [3]:
supply = pd.DataFrame([
    ['G1', 'B1', 90., 12.],
    ['G2', 'B1', 80., 20.],
], columns=['id', 'node', 'capacity (MW)', 'offer ($/MW)']).set_index('id')
assert supply.node.isin(nodes).all()
supply

,node,capacity (MW),offer ($/MW)
id,,,
G1,B1,90.0,12.0
G2,B1,80.0,20.0


In [4]:
demand = pd.DataFrame([
    ['D1', 'B1', 100., 40.],
    ['D2', 'B1', 50., 35.],
], columns=['id', 'node', 'demand (MW)', 'bid ($/MW)']).set_index('id')
assert demand.node.isin(nodes).all()
demand

,node,demand (MW),bid ($/MW)
id,,,
D1,B1,100.0,40.0
D2,B1,50.0,35.0


In [5]:
coalitions = list(iter.product(list(iter.product([False, True], [False, True])),
                               list(iter.product([False, True], [False, True]))))

In [6]:
def solve(supply, demand):
    with Model('power') as M:
        nsupply = len(supply)
        ndemand = len(demand)
        
        if (nsupply == 0) or (ndemand == 0):
            return Result(demand, supply, np.nan, 0)

        M = Model('power')
        pD = M.variable('pD', ndemand, Domain.inRange(0, demand['demand (MW)'].values))
        pG = M.variable('pG', nsupply, Domain.inRange(0, supply['capacity (MW)'].values))
        bidD = demand['bid ($/MW)'].values
        offerG = supply['offer ($/MW)'].values

        balance = M.constraint('balance', Expr.sub(Expr.sum(pD), Expr.sum(pG)), Domain.equalsTo(0.))
        obj = Expr.sub(Expr.dot(bidD, pD), Expr.dot(offerG, pG))
        M.objective('obj', ObjectiveSense.Maximize, obj)
        M.solve()

        price = balance.dual()[0]
        supply['supplied (MW)'] = pG.level()
        supply['revenue'] = pG.level() * price
        supply['profit'] = pG.level() * (price - offerG)
        demand['consumed (MW)'] = pD.level()
        demand['payment'] = pD.level() * price
        demand['profit'] = pD.level() * (bidD - price)
        welfare = M.primalObjValue()
        return Result(supply, demand, price, welfare)

In [7]:
result = solve(supply, demand)

In [8]:
display(result.supply)
display(result.demand)
print('Clearing price: {:g}'.format(result.price))
print('Energy scheduled: {:g}'.format(result.demand['consumed (MW)'].sum()))
print('Social welfare: {:g}'.format(result.welfare))

,node,capacity (MW),offer ($/MW),supplied (MW),revenue,profit
id,,,,,,
G1,B1,90.0,12.0,90.0,1799.999984,719.999984
G2,B1,80.0,20.0,60.0,1199.999989,-0.000011


,node,demand (MW),bid ($/MW),consumed (MW),payment,profit
id,,,,,,
D1,B1,100.0,40.0,100.0,1999.999982,2000.000018
D2,B1,50.0,35.0,50.0,999.999991,750.000009


Clearing price: 20
Energy scheduled: 150
Social welfare: 3470


In [9]:
values = [solve(supply.loc[list(sidx)].copy(),
                demand.loc[list(didx)].copy()).welfare 
          for sidx, didx in coalitions]

In [10]:
pd.DataFrame([x+y+(z,) for (x, y), z in zip(coalitions, values)],
             columns=list(supply.index)+list(demand.index)+['value'])

,G1,G2,D1,D2,value
0,False,False,False,False,0.0
1,False,False,False,True,0.0
2,False,False,True,False,0.0
3,False,False,True,True,0.0
4,False,True,False,False,0.0
5,False,True,False,True,750.0
6,False,True,True,False,1600.0
7,False,True,True,True,1600.0
8,True,False,False,False,0.0
9,True,False,False,True,1150.0


In [11]:
M = Model('excess')
payoff = M.variable('payoff', len(supply) + len(demand), Domain.greaterThan(0.))
max_excess = M.variable('max_excess', 1)

balance = M.constraint(Expr.sub(values[-1], Expr.sum(payoff)), Domain.equalsTo(0.))

for (sidx, didx), value in zip(coalitions, values):
    payoff_idx = np.flatnonzero(sidx+didx).astype('int32')
    if len(payoff_idx) == 0:
        continue
    excess = Expr.sub(value, Expr.sum(payoff.pick(payoff_idx)))
    M.constraint(Expr.sub(max_excess, excess), Domain.greaterThan(0.))

M.objective('obj', ObjectiveSense.Minimize, max_excess)
M.solve()

print('Max excess: {:g}'.format(M.primalObjValue()))
supply['payoff'] = payoff.level()[0:len(supply)]
demand['payoff'] = payoff.level()[len(supply):]

display(supply.round(1))
display(demand.round(1))


Max excess: 1.59616e-10


,node,capacity (MW),offer ($/MW),supplied (MW),revenue,profit,payoff
id,,,,,,,
G1,B1,90.0,12.0,90.0,1800.0,720.0,1150.0
G2,B1,80.0,20.0,60.0,1200.0,-0.0,950.0


,node,demand (MW),bid ($/MW),consumed (MW),payment,profit,payoff
id,,,,,,,
D1,B1,100.0,40.0,100.0,2000.0,2000.0,1370.0
D2,B1,50.0,35.0,50.0,1000.0,750.0,-0.0
